In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
#r ".\binaries\ApplicationWithIDT.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using ApplicationWithIDT;
using XESF;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


# Run Simulation

In [2]:
var date = DateTime.Now.ToString("ddMMyyyy");
date

30082023

open a database

In [3]:
string dbPath = @"WedgeFlow_db_" + date ;
var database = OpenOrCreateDatabase(dbPath);

Creating database 'WedgeFlow_db_30082023'.


run WedgeFlow simulation and safe to database

In [5]:
BoSSS.Solution.Application.InitMPI();
BoSSS.Solution.Application.DeleteOldPlotFiles();
var p = new XESFMain();

var C = XESFHardCodedControl.XDGWedgeFlow_TwoLs_Base(
    dbPath:dbPath,
    optiLSDegree: 1,
    lsDegree: 1,
    shocksetup: ApplicationWithIDT.GetLevelSet.FromFunction,
    optiLevelSetType: OptiLevelSetType.SplineLevelSet,
    initialValue: ApplicationWithIDT.GetInitialValue.FromFunctionPerSpecies,
    MaxIterations: 200,
    dgDegree: 0,
    numOfCellsX: 15,
    numOfCellsY: 10,
    initialAngle_shockLS: 32,
    PlotInterval: -1,
    interfaceFluxLS2: XESF.Fluxes.ConvectiveInterfaceFluxes.GodunovInterface,
    bulkFlux: XESF.Fluxes.ConvectiveBulkFluxes.OptimizedHLLC,
    agg: 0.4
);
p.Init(C);
p.RunSolverMode();


rm;
Grid Edge Tags changed.
Session ID: 88f743c3-0474-4e8e-953a-c4a6bea331fb, DB path: 'WedgeFlow_db_30082023'
Session directory 'WedgeFlow_db_30082023\sessions\88f743c3-0474-4e8e-953a-c4a6bea331fb'.
Initial Value : l2: |r|= 1.0778378E+01, |f_err|=2.4570273E+01, | f_phi |=0E+00,

Starting Opt-Iter No.1: 
alpha, ||R1||  1, Error 
alpha, ||R1||  0.5, Error 
alpha, ||R1||  0.25, Error 
alpha, M, predM  0.125, 10770.664450276225, 12304.91412764366
l2: ||R0||= 9.0972954E+00, f_err=2.150577E+01, f_phi=0E+00, Alpha = 1.25E-01, Gamma = 1E+00, Kappa = 1E+03, Mu = 1.63E+02, QuadOrder=2
**NOT** terminating now, minimal number of iterations not reached


Starting Opt-Iter No.2: 
alpha, ||R1|| 1 LevelSetCFL 
alpha, ||R1|| 0.5 LevelSetCFL 
alpha, ||R1||  0.25, Error 
alpha, ||R1||  0.125, Error 
alpha, ||R1||  0.0625, Error 
alpha, M, predM  0.03125, 5648.869840730589, 5835.951678368398
l2: ||R0||= 8.8168796E+00, f_err=2.098005E+01, f_phi=0E+00, Alpha = 3.13E-02, Gamma = 1E+00, Kappa = 1E+03, Mu = 8

# Postprocessing

we need to load something from the ApplicationWithIDT assembly 

In [ ]:
Assembler aaa = new Assembler(3);
var si = database.Sessions.Pick(0);

## Enthalpy error

enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(c_P -1)(\rho E - k),~k=c_P \text{Ma}^2\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$

For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

As we have uniform inflow the v-velocity is zero and we have
$$ h_{in} = \frac{\gamma}{\gamma-1} + \frac{1}{2}u_{in}^2 $$
We prescribe the inflow conditions in terms of pressure $p_{in}=1$, density $\rho_{in}=1$ and the Mach number $M_{in}=2$. From those the velocity is computed as 
$$ u_{in} = M_{in} \sqrt{\gamma \frac{p_{in}}{\rho_{in}}}= 2\sqrt{\gamma}$$

In [ ]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = (XDGField) si.Timesteps.Last().GetField("h");
var enthalpy_exact = (XDGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 2;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

In [ ]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.GetSpeciesShadowField("L").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.GetSpeciesShadowField("R").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2NormAllSpecies();

In [ ]:
p.GetResEnthalpyPlot(si,EEN).dataGroups[2].Values.SaveToTextFile("siWF2EnthalpyErrors.txt");
p.GetResEnthalpyPlot(si,EEN).ToGnuplot().PlotSVG(xRes:1200,yRes:500)

In [ ]:
p.PlotShadowFields(si);